In [6]:
#allow output from every line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from pybaseball import lahman
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import seaborn as sns
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objects as go
from dash.dependencies import Input, Output

In [7]:
#load data
batting_data = pd.read_csv('Batting_Data.csv') #loads scraped data in csv format

In [8]:
# Build App

#radar average stuff
categories = ["H", "2B", "3B", "HR", "SLG", "RBI", "OBP"] #creates list of variables to be considered in radar plot
fig1=go.Figure() #creates blank figure for the radar plot
fig2=go.Figure() #creates blank figure for the scatter plot
fig1.add_trace(go.Scatterpolar( #initializes radar plot for league averages
    r=[0.5, 0.5,0.5, 0.5, 0.5, 0.5, 0.5], #list of league average percentiles
    theta=categories,#sets list of variables to radar plot
    fill='toself', #visual effects for radar plot
    name="League Average")) #names section of radar plot
fig1.update_layout( #updating radar plot
    polar=dict( #polar plot
        radialaxis=dict( 
            visible=False, #hides radial axes
            range=[0,1])), #sets radar plot range from 0-1 as all percentiles fall in these values
    showlegend=True) #shows legend on radar plot
 
fig3=go.Figure() #creates blank figure for the pie chart
#formatting stuff
app = JupyterDash(__name__)#creates app structure 
app.layout = html.Div([
    html.H1("SAL384 Project: Player Comparison App"), #creates title of app
    html.Label([
        "Player:", #creates prompt for name input
        dcc.Input( #creates name input box
            id='name_input', type='text',
            placeholder='Enter Player Name'
            )
    ]),
    html.P('Radar Plot: Comparing 7 player batting statistic percentiles vs. league average (excl. Pitchers).'), #description for radar plot
    html.P('Pie Chart: Displaying player hit type distribution.'), #description for pie chart
    html.P('Scatter Plot: Comparing player slugging percentage against runs scored. Player selected is highlighted.'), #description for scatter plot
        dcc.Graph(figure=fig1, id="fig1", style={'display':'inline-block', 'width':'40%'}),#setting radar plot into app, spacing 
        dcc.Graph(figure=fig3, id="fig3", style={'display': 'inline-block', 'width':'40%'}),#setting pie chart into app, spacing
        dcc.Graph(figure=fig2, id="fig2", #setting scatter plot, spacing
             config={ #setting options on scatter plot to make it more detailed and interactive 
                 'staticPlot':False,
                 'scrollZoom':True,
                 'doubleClick':'reset',
                 'showTips':True,
                 'displayModeBar':True,
                 'watermark':True
             })
])


@app.callback( #callback for radar plot
    Output('fig1', 'figure'), #output is radar plot as a figure
    [Input("name_input", "value")] #takes name input to update radar plot
)

def update_figure(Player): #function to update radar plot
    fig1=go.Figure() #sets blank figure to figure 1 for radar plot
    fig1.add_trace(go.Scatterpolar( #double checking that league averages are implemented on radar plot
        r=[0.5, 0.5,0.5, 0.5, 0.5, 0.5, 0.5], #league average percentile values
        theta=categories,#implementing categories
        fill='toself', #visual effect
        name="League Average")) #naming average plot
    fig1.update_layout(
        polar=dict(
        radialaxis=dict(
        visible=False, #hides radial axes
        range=[0,1])), #sets radar plot range from 0-1 as all percentiles fall in these values
        showlegend=True) #shows legend for league averages
    if(len(batting_data[batting_data['Name'] == Player]) > 0): #makes sure no plots are implemented unless a player's full name is entered
        x_row=batting_data[batting_data['Name'] == Player].index[0] #gets index of player's row
        player_hits=batting_data['H_percentile'][x_row] #gets players' hits percentile
        player_doubles=batting_data['2B_percentile'][x_row] #gets players' doubles percentile
        player_triples=batting_data['3B_percentile'][x_row] #gets players' triples percentile
        player_hrs=batting_data['HR_percentile'][x_row] #gets players' home runs percentile
        player_SLG=batting_data['SLG_percentile'][x_row] #gets players' slugging percentile
        player_RBI=batting_data['RBI_percentile'][x_row] #gets players' RBI percentile
        player_OBP=batting_data['OBP_percentile'][x_row] #gets players' OBP percentile
        fig1.add_trace(go.Scatterpolar( #adding player plot over league average 
            r=[player_hits, player_doubles, player_triples, player_hrs, player_SLG,
              player_RBI, player_OBP],#adding player values
            theta=categories, #maintaining categories
            fill='toself', #visual effect
            name=Player)) #names plot after entered player
    return(fig1) #returns updated radar plot

#Define callback to update graph
@app.callback( #callback for scatter plot
    Output('fig2', 'figure'), #outputs scatter plot as a figure
    [Input("name_input", "value")] #takes input of player name to update scatter plot
)

def update_scatter(Player): #function to update scatter plot
    fig2=go.Figure() 
    dff=batting_data
    if(len(batting_data[batting_data['Name'] == Player]) > 0): #makes sure no plots are implemented unless a player's full name is entered
        x_row=batting_data[batting_data['Name'] == Player].index[0] #gets index of player's row
        batting_data["Ind"] = (batting_data["Name"] == Player).astype(str)#creates column as a player indicator, used for highlighting in the scatter plot
        fig2=px.scatter(batting_data, x='SLG', y='R', hover_data=["Name", "Team", "Pos"], color="Ind") #creates scatter plot and highlights player selected, also updates hover data
        fig2.update_layout(showlegend=False) #hides legend
    return(fig2) #returns scatter plot updated

@app.callback( #callback for pie chart
    Output('fig3', 'figure'), #returns pie chart as a figure
    [Input("name_input", "value")] #takes input of player name to update pie chart
)

def update_pie(Player): #function to update pie chart
    fig3=go.Figure()
    if(len(batting_data[batting_data['Name'] == Player]) > 0):  #makes sure no plots are implemented unless a player's full name is entered
        y_row=batting_data[batting_data['Name'] == Player].index[0] #gets index of player's row
        labels=['Singles', 'Doubles', 'Triples', 'Home Runs'] #selecting variables to consider in pie chart as hitting distribution
        values= [batting_data['Singles'][y_row], batting_data['2B'][y_row], batting_data['3B'][y_row], batting_data['HR'][y_row]] #set player values to pie chart
        fig3=go.Figure(data=[go.Pie(labels=labels, values=values)]) #creating pie chart
    return(fig3) #returns created pie chart
app.run_server(debug=True, use_reloader=False) 

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.